In [2]:
from functools import reduce
from collections import deque

## Задание №1
```
📌 Создайте класс-функцию, который считает факториал числа при вызове экземпляра.
📌 Экземпляр должен запоминать последние k значений.
📌 Параметр k передаётся при создании экземпляра.
📌 Добавьте метод для просмотра ранее вызываемых значений и их факториалов.
```

In [3]:
class Fact:
    def __init__(self, *, k):
        self.buf = deque(maxlen=k)

    def __call__(self, n):
        res = reduce(lambda f, i: f * i, range(2, n + 1), 1)
        self.buf.append(res)
        return res

    def __str__(self):
        return ", ".join(map(str, self.buf))

In [4]:
f = Fact(k=3)

print(f"{f(4) = }")
print(f"{f(2) = }")
print(f"{f(1) = }")
print(f"{f(5) = }")
print(f"{f(7) = }")
print(f)

f(4) = 24
f(2) = 2
f(1) = 1
f(5) = 120
f(7) = 5040
1, 120, 5040


In [ ]:
class Faktorial:
    storage = []
    def __init__(self, k):
        self.k = k

    def __str__(self):
        return '\n'.join(f'{i[0]}! = {i[1]}' for i in self.storage)

    def __call__(self, value):
        faktorial = 1
        for i in range(value):
            faktorial *= i+1
        if len(self.storage) == self.k:
            self.storage.pop(0)
        self.storage.append((value, faktorial))
        return faktorial

## Задание №2
```
📌 Доработаем задачу 1.
📌 Создайте менеджер контекста, который при выходе сохраняет значения в JSON файл.
```

In [5]:
import json

class FactJson(Fact):
    def __init__(self, fName, *, k):
        super().__init__(k=k)
        self.fName = fName

    def __enter__(self):
        return self

    def __exit__(self, *args):
        with open(self.fName, "w") as jsonFile:
            json.dump(list(self.buf), jsonFile)

In [6]:
with FactJson("task4.json", k=3) as f:
    print(f"{f(3) = }")
    print(f"{f(2) = }")
    print(f"{f(5) = }")
    print(f"{f(4) = }")
    print(f"{f(1) = }")

f(3) = 6
f(2) = 2
f(5) = 120
f(4) = 24
f(1) = 1


## Задание №3
```
📌 Создайте класс-генератор.
📌 Экземпляр класса должен генерировать факториал числа в диапазоне от start до stop с шагом step.
📌 Если переданы два параметра, считаем step=1.
📌 Если передан один параметр, также считаем start=1.
```

In [22]:
class FactGen(Fact):
    def __init__(self, start, stop = None, step = 1, *, k):
        super().__init__(k=k)
        if stop:
            self.start = start
            self.stop = stop
        else:
            self.start = 1
            self.stop = start
        self.step = step

    def __iter__(self):
        self.i = self.start
        return self
    
    def __next__(self):
        if self.i < self.stop:
            res = self.__call__(self.i)
            self.i += self.step
            return res
        raise StopIteration

Возврат по yield:

In [26]:
class FactGen(Fact):
    def __init__(self, start, stop = None, step = 1, *, k):
        super().__init__(k=k)
        if stop:
            self.start = start
            self.stop = stop
        else:
            self.start = 1
            self.stop = start
        self.step = step

    def __iter__(self):
        return self.__next__()
    
    def __next__(self):
        for i in range(self.start, self.stop, self.step):
            yield self.__call__(i)
        self.start = self.stop # для исключения повторных итераций

упрощённо:

In [9]:
class FactGen(Fact):
    def __init__(self, start, stop = None, step = 1, *, k):
        super().__init__(k=k)
        if stop:
            self.start = start
            self.stop = stop
        else:
            self.start = 1
            self.stop = start
        self.step = step

    def __iter__(self):
        for i in range(self.start, self.stop, self.step):
            yield self.__call__(i)
        self.start = self.stop # для исключения повторных итераций

In [27]:
g = FactGen(1, 5, k=3)
for f in g:
    print(f)

for f in g:
    print(f)

print(g)

1
2
6
24
2, 6, 24


In [ ]:
class Faktorial:

    def __init__(self, *args):
        self.step = 1
        self.start = 1
        self.stop = args[0]
        if len(args) > 1:
            self.start = args[0]
            self.stop = args[1]
            if len(args) > 2:
                self.step = args[2]
        self.faktorial = 1
        self.num = 0

    def __iter__(self):
        return self

    def __next__(self):
        while self.num < self.stop:
            self.num += 1
            self.faktorial *= self.num
            if self.start <= self.num < self.stop and (self.num - self.start) % self.step == 0:
                return self.faktorial
        raise StopIteration


f = Faktorial(2, 7, 2)
for num in f:
    print(num)

## Задание №4
```
📌 Доработайте класс прямоугольник из прошлых семинаров.
📌 Добавьте возможность изменять длину и ширину прямоугольника и встройте контроль недопустимых значений (отрицательных).
📌 Используйте декораторы свойств.
```

In [1]:
class Rectangle:
    def __init__(self, width: int, height: int=None) -> None:
        self._dimX = width
        self._dimY = height if height else width
    
    @property
    def width(self):
        return self._dimX

    @property
    def height(self):
        return self._dimY
    
    def perimeter(self) -> int:
        return (self._dimX + self._dimY) * 2
    
    def area(self) -> int:
        return self._dimX * self._dimY
    
    def __add__(self, other):
        return self.__class__(self._dimX + other._dimX, self._dimY + other._dimY)
    
    def __sub__(self, other):
        return self.__class__(abs(self._dimX - other._dimX), abs(self._dimY - other._dimY))
    
    def __lt__(self, other) -> bool:
        return self.area() < other.area()
    
    def __eq__(self, other) -> bool:
        return self.area() == other.area()
    
    def __le__(self, other) -> bool:
        return self.area() <= other.area()
    
    def __str__(self) -> str:
        return f"Прямоугольник со сторонами {self._dimX} и {self._dimY}"
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self._dimX}, {self._dimY})"

In [7]:
class RectangleMod(Rectangle):
    __slots__ = ('_dimX', '_dimY')
    
    @property
    def width(self):
        return self._dimX

    @property
    def height(self):
        return self._dimY
    
    @width.setter
    def width(self, value):
        if not isinstance(value, int) or value <= 0:
            raise ValueError("Некорректное значение ширины")
        self._dimX = value
    
    @height.setter
    def height(self, value):
        if not isinstance(value, int) or value <= 0:
            raise ValueError("Некорректное значение высоты")
        self._dimY = value

In [8]:
rect = RectangleMod(20, 30)

rect.Z = 10
rect.width = 25
print(rect)
rect.__dict__

Прямоугольник со сторонами 25 и 30


{'Z': 10}

## Задание №5
```
📌 Доработаем прямоугольник и добавим экономию памяти для хранения свойств экземпляра без словаря __dict__.
```

## Задание №6
```
📌 Изменяем класс прямоугольника.
📌 Заменяем пару декораторов проверяющих длину и ширину на дескриптор с валидацией размера.
```

In [4]:
class Dim:
    def __set_name__(self, owner, name):
        self.attrName = owner.ATTR_NAMES.get(name, ("_" + name,))[0]
        self.desc = owner.ATTR_NAMES.get(name, (None, name))[1]
    
    def __get__(self, instance, owner):
        return getattr(instance, self.attrName)
    
    def __set__(self, instance, value):
        if not isinstance(value, int) or value <= 0:
            raise ValueError(f"Некорректное значение размера: {self.desc}")
        setattr(instance, self.attrName, value)

In [22]:
class RectangleMod1(Rectangle):
    ATTR_NAMES = {
        'width': ('_dimX', "ширина"),
        'height': ('_dimY', "высота"),
    }
    
    __slots__ = ('_dimX', '_dimY')
    width = Dim()
    height = Dim()

    def __init__(self, width: int, height: int=None) -> None:
        self.width = width
        self.height = height if height else width

In [33]:
rect1 = RectangleMod1(20, 30)
rect2 = RectangleMod1(3, 7)

rect3 = rect1 + rect2
rect3.width = 25
print(rect1.height)
print(repr(rect3))

30
RectangleMod1(25, 37)


In [ ]:
def check_for_zero(x, y):
    while x <= 0:
        if y > 1:
            y -= 1
            x += 1
        else:
            y /= 2
            x = y
    return x, y

class Range:
    def __init__(self, min_value: int = None, max_value: int = None):
        self.min_value = min_value
        self.max_value = max_value

    def __set_name__(self, owner, name):
        self.param_name = '_' + name

    def __get__(self, instance, owner):
        return getattr(instance, self.param_name)

    def __set__(self, instance, value):
        self.validate(value)
        setattr(instance, self.param_name, value)

    def __delete__(self, instance):
        raise AttributeError(f'Свойство "{self.param_name}" нельзя удалять')

    def validate(self, value):
        if not isinstance(value, int):
            raise TypeError(f'Значение {value} должно быть целым числом')
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'Значение {value} должно быть больше или равно {self.min_value}')
        if self.max_value is not None and value >= self.max_value:
            raise ValueError(f'Значение {value} должно быть меньше {self.max_value}')


class Rectangle:
    '''Класс Прямоугольник.
    Два свойства - длина и ширина.
    Методы вычисления периметра и площади, сложения, вычитания, сравнения и вывода в консоль'''
    __slots__ = ('_width', '_height')

    width = Range(1)
    height = Range(1)
    def __init__(self, width, height=None):
        self.width = width
        if height == None:
            self.height = self.width
        else:
            self.height = height
        # if self.width <= 0 or self.height <= 0:
        #     raise ValueError('Длины сторон должны быть больше нуля')

    def perimeter(self):
        '''Метод вычисления периметра параллелограмма'''
        return 2*(self.height+self.width)

    def area(self):
        '''Метод вычисления площади прямоугольника'''
        return self.height*self.width

    def __add__(self, other):
        '''Сложение параллелограммов путём сложения их периметров'''
        height = self.height + other.height
        width = self.width + other.width
        return Rectangle(width, height)

    def __sub__(self, other):
        '''Вычитание параллелограммов путём вычитания их периметров
           с исключением отрицательных значений'''
        width = abs(self.width - other.width)
        perimeter = abs(self.perimeter() - other.perimeter())
        if perimeter == 0:
            width, height = 0, 0
        else:
            height = perimeter // 2 - width
            width, height = check_for_zero(width, height)
            height, width = check_for_zero(height, width)
        return Rectangle(width, height)

    def __eq__(self, other):
        '''Сравнение фигур путём сравнения их площадей'''
        return self.area() == other.area()

    def __gt__(self, other):
        '''Сравнение фигур путём сравнения их площадей'''
        return self.area() > other.area()

    def __lt__(self, other):
        '''Сравнение фигур путём сравнения их площадей'''
        return self.area() < other.area()

    def __ne__(self, other):
        '''Сравнение фигур путём сравнения их площадей'''
        return self.area() != other.area()

    def __ge__(self, other):
        '''Сравнение фигур путём сравнения их площадей'''
        return self.area() >= other.area()

    def __le__(self, other):
        '''Сравнение фигур путём сравнения их площадей'''
        return self.area() <= other.area()

    def __str__(self):
        '''Магический метод.
        Возвращает строку, описывающую ширину и высоту прямоугольника в виде:
        Прямоугольник со сторонами 2 и 3,
        где первое число - это ширина, а второе - высота.
        '''
        return f'Прямоугольник со сторонами {self.width} и {self.height}'

    def __repr__(self):
        return f'Rectangle({self.width}, {self.height})'